<a href="https://colab.research.google.com/github/ahsanrazi/LangChain/blob/main/03_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.3 MB/s eta 0:00:00


In [2]:
!pip install -qU langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.4 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
gemini_api_key = userdata.get("GEMINI_API_KEY").strip()

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp", api_key=gemini_api_key)

In [6]:
# First use the model directly

from langchain_core.messages import HumanMessage

model.invoke([HumanMessage("Hi I am Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-13a16c17-1b0d-430c-875a-acbf716a5bae-0', usage_metadata={'input_tokens': 5, 'output_tokens': 19, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

In [7]:
# The model on its own does not have any concept of state.

model.invoke([HumanMessage("What is my name?")])

AIMessage(content="I am an AI, and I don't have access to your personal information, so I don't know your name. You haven't told me what your name is. \n\nIf you'd like me to call you something, you can tell me what you'd prefer!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-7b1ba28e-3ba2-47c5-bd02-098a32c59513-0', usage_metadata={'input_tokens': 6, 'output_tokens': 61, 'total_tokens': 67, 'input_token_details': {'cache_read': 0}})

In [8]:
# To get around this, we need to pass the entire conversation history into the model.

from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! You told me that at the beginning of our conversation. 😊', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-35be26f9-9587-4a6f-b1f4-e66b658d165d-0', usage_metadata={'input_tokens': 25, 'output_tokens': 18, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}})

# Message persistence

In [9]:
# LangGraph implements a built-in persistence layer, making it ideal for chat applications that support multiple conversational turns.

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [10]:
# We now need to create a config that we pass into the runnable every time.

config = {"configurable": {"thread_id": "abc123"}}

In [11]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [12]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Bob, it's nice to meet you! I'm an AI, but you can think of me as a helpful assistant. What can I do for you today?


In [13]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [14]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! You told me that at the beginning of our conversation. 😊


In [15]:
# Our chatbot now remembers things about us. If we change the config to reference a different thread_id, we can see that it starts the conversation fresh.

config = {"configurable": {"thread_id": "xyz987"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [16]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

As a large language model, I don't have access to personal information about you. Therefore, I don't know your name. You haven't told me what it is.

If you'd like to share it, I'd be happy to address you by your name! 😊


# Prompt templates

In [17]:
# Right now, all we've done is add a simple persistence layer around the model.
# We can start to make the chatbot more complicated and personalized by adding in a prompt template.

# Prompt Templates help to turn raw user information into a format that the LLM can work with.
# To add in a system message, we will create a ChatPromptTemplate. We will utilize MessagesPlaceholder to pass all the messages in.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [18]:
# We can now update our application to incorporate this template

workflow = StateGraph(MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [19]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)

In [20]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ahoy there, Jim! A fine name ye have, says I! What brings ye to me ol' ship today? Are ye lookin' for treasure, tales, or somethin' else entirely?


In [21]:
# Let's now make our prompt a little bit more complicated

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [22]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [23]:
config = {"configurable": {"thread_id": "abc456"}}

In [24]:
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)

In [25]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola Bob! Es un placer conocerte. ¿En qué puedo ayudarte hoy?


In [26]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob.


# Managing Conversation History

In [27]:
# One important concept to understand when building chatbots is how to manage conversation history.
# If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM.

# Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.



In [71]:
# LangChain comes with a few built-in helpers for managing a list of messages.
# In this case we'll use the trim_messages helper to reduce how many messages we're sending to the model.

# The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message
# and whether to allow partial messages.


from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [64]:
# Let's now make our prompt a little bit more complicated

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [65]:
# To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)


def call_model(state: State):
    # trimmed_messages = trimmer.invoke(state["messages"])
    trimmed_messages = trimmer.invoke([m for m in state["messages"] if not isinstance(m, SystemMessage)])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [66]:
config = {"configurable": {"thread_id": "abc879"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)

In [67]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. You told me that at the beginning. 😊


In [68]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)

In [69]:
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked "what's 2 + 2".


# Streaming

In [ ]:
# LLMs can sometimes take a while to respond, and so in order to improve the user experience
# one thing that most applications do is stream back each token as it is generated.

In [73]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream({"messages": input_messages, "language": language}, config, stream_mode="messages",):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Okay| Todd, here's another one for you:

Why did the bicycle fall| over? 

Because it was two tired!|